# Start the Project 🤩


## Step One 
### Exploratory Data Analysis
**Before Data Preprocessing**

-  Image Count & Class Distribution

-  Random Image Visualization

-  Image Shape / Size / Channels Check

- File Format Check (.jpg / .png / etc.)

- Missing / Corrupt / Blank Images

## Step Two
###  Data Preprocessing

- Image Collection  ✅

 Collect all image folders, combine them into one main folder, and assign clear, correct labels for each. Make sure everything is well organized! ✅🗂️🧠

  - Example labels like this
   
    8-cell Grade A, 8-cell Grade B, 8-cell Grade C, Morula Grade A, Morula Grade B, Morula Grade C, Blastocyst Grade A, Blastocyst Grade B, Blastocyst Grade C, Error Images

- Make sure all the images are in standard format like .jpg.

- Resize

- Convert RGB 

- Data Augmentation (Each folder have 1000 images)

- convert to array 

- Check Duplicate values 

- Split Data (train, test)

- Save the train and test 

## Step Three
###  After Data preprocessing 

Basic check 
- shape 
- Duplicate 
- Missing values
- Range 0 to 1 
- RGB
- Output count balanced or not

# Step Four

### Model Building 

##### update to the MLflow 

- 1. ConvNext
- 2. DenseNet201
- 3. Efficient_Net_B7
- 4. Res_Net_152
- 5. Swin_Transformer
- 6. Efficient VIT


# Step Five

**Check Which Model Gives Highest Accuracy**

### In my case, **Efficient VIT** gives the highest accuracy:  
##### **Train Accuracy** – 99%  
##### **Test Accuracy** – 95%


# Step Six

### **Model Evaluation Step**

#### - ✅ Precision  
#### - ✅ Recall  
#### - ✅ Cross Validation  
#### - ✅ Check with new images – does the model give correct results or not?

# Model building 


# model training phase 





In [19]:
%pwd

'a:\\Artificial intelligence\\Intership works\\Embryo Project\\Embryo-Project-DeepLearning'

In [3]:
import os

os.chdir('../')

In [20]:
%pwd

'a:\\Artificial intelligence\\Intership works\\Embryo Project\\Embryo-Project-DeepLearning'

In [21]:
# Update entity 

from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class EvaluationConfig:
    root_dir: Path
    trained_model: Path
    onnx_model_path32: Path
    onnx_model_path16: Path
    test_data: Path


In [22]:

# Update config manager in `src/config`  
from EmbryoQualityCheck.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from EmbryoQualityCheck.utils.common import read_yaml, create_directories 


class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        """
            read the config and params 
            and create directories
        """
        # config
        self.config = read_yaml(config_filepath)
        # params
        self.params = read_yaml(params_filepath)
        # create dir (main)
        create_directories([self.config.artifacts_root])

    """
        Model Evaluation 
    """
    def get_evaluation_config(self) -> EvaluationConfig:
        # training 
        onnx_model = self.config.onnx_model

        create_directories([onnx_model.root_dir])

        eval_config = EvaluationConfig(
            root_dir=Path(onnx_model.root_dir),
            trained_model= Path(onnx_model.trained_model),
            onnx_model_path32=Path(onnx_model.onnx_model_path32),
            onnx_model_path16=Path(onnx_model.onnx_model_path16),
            test_data=Path(onnx_model.test_data),
        )   
        
        return eval_config

In [ ]:
# 6. Update the components 

# 🧠 Core ML Libraries
import torch
import timm
import onnx
from onnxconverter_common import float16
import onnxruntime as ort

# 🖼️ Vision & Transforms
from PIL import Image
import torchvision.transforms as transforms

# 📊 Metrics
from sklearn.metrics import accuracy_score

# 🔁 Progress Bar
from tqdm import tqdm

# 🌐 System & OS
import os
import numpy as np

# ⚙️ Custom Modules
from EmbryoQualityCheck import logger


class Evaluation:

    def __init__(self, config: EvaluationConfig):
        self.config = config

    def convert_to_onnxModel(self):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # 🧠 Load the model
        model = timm.create_model('efficientvit_b2.r288_in1k', pretrained=False)
        model.to(device)
        
        model.load_state_dict(torch.load(self.config.trained_model))
        model.eval()

        # 📦 Dummy input
        dummy_input = torch.randn(1, 3, 288, 288).to(device)

        # 🟠 Export to ONNX (FP32)
        torch.onnx.export(
            model,
            dummy_input,
            self.config.onnx_model_path32,
            input_names=['input'],
            output_names=['output'],
            opset_version=11,
            export_params=True,
            do_constant_folding=True
        )

        # ⚒️ Convert to FP16
        model_fp32 = onnx.load(self.config.onnx_model_path32)
        model_fp16 = float16.convert_float_to_float16(model_fp32)
        onnx.save(model_fp16, self.config.onnx_model_path16)

        logger.info("ONNX FP16 model saved as efficientvit_embryo_fp16.onnx")
    
    # test data evaluation 
    def TestEvaluation(self):
        predict = []
        y_true = []

        root_path = self.config.test_data


        # Load ONNX model
        session = ort.InferenceSession(self.config.onnx_model_path16)

        # Define transform
        transform = transforms.Compose([
            transforms.Resize((288, 288)),
            transforms.ToTensor(),
        ])
        
        # Class names
        class_names = [
            '8-cell Grade A', '8-cell Grade B', '8-cell Grade C',
            'Blastocyst Grade A', 'Blastocyst Grade B', 'Blastocyst Grade C',
            'Error Images', 'Morula Grade A', 'Morula Grade B', 'Morula Grade C'
        ]

        for folder in tqdm(os.listdir(root_path), desc="Processing folders"):
            folder_path = os.path.join(root_path, folder)
            
            for img_name in os.listdir(folder_path):
                image_path = os.path.join(folder_path, img_name)

                # Load and transform image
                img = Image.open(image_path).convert("RGB")
                img_tensor = transform(img).unsqueeze(0).numpy().astype(np.float16)

                # Run inference
                outputs = session.run(None, {"input": img_tensor})
                probs = outputs[0]
                pred_class = np.argmax(probs)

                # Store prediction and ground truth
                predict.append(class_names[pred_class])
                y_true.append(folder)  # True label from folder name

        logger.info(f'Accuracy {accuracy_score(y_true, predict)}')

    

In [ ]:
# 7. Update the pipeline 

try:
    config = ConfigurationManager()
    evaluation_config = config.get_evaluation_config()
    evaluation = Evaluation(evaluation_config)
    evaluation.convert_to_onnxModel()
    evaluation.TestEvaluation()
except Exception as e:
    raise e 


[2025-04-25 19:55:45,000: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-25 19:55:45,004: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-25 19:55:45,005: INFO: common: created directory at: artifacts]
[2025-04-25 19:55:45,007: INFO: common: created directory at: artifacts/onnx_model]
ji


Processing folders: 100%|██████████| 10/10 [01:32<00:00,  9.22s/it]

[2025-04-25 19:57:17,388: INFO: 872567979: Accuracy 0.9645464025026069]
